# Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import preds file

In [ ]:
ECD_predictors=pd.read_csv('RawDataPredictors/ECD.csv')
successful_predictors=pd.read_csv('RawDataPredictors/Successful_Readings.csv')
unsuccessful_predictors=pd.read_csv('RawDataPredictors/Unsuccessful_Readings.csv')

# Import raw time series file

In [ ]:
ECD_ts=pd.read_csv('TimeSeriesData/ECDTS/ECD_TS.csv')
ECD_synthetic_ts=pd.read_csv('TimeSeriesData/ECDTS/ECD_TS_Synthetic.csv')
unsuccessful_ts=pd.read_csv('TimeSeriesData/UnsuccessfulReadingsTS/US_TS.csv')

# Filtering

### Frequency = 1/T

**Low frequency ==> Large time period ==> less oscillations in a unit time** <br>
**High frequency ==> Small time period  ==> more oscillations in a unit time** <br>

# Waveforms of unsuccessful data

In [ ]:
plt.style.use('dark_background')
plt.figure(figsize=(20,10))
data=np.array(unsuccessful_ts.iloc[0,1:])
x = np.arange(0.2, 300.2, 0.2)
plt.plot(x,data)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel('Time (seconds)', fontsize=15)
plt.ylabel('Current (nA)', fontsize=15)
plt.title('Example waveform of unsuccessful data', fontsize=25)
plt.show()

### Zooming on to the sensor's noise

In [ ]:
plt.style.use('default')
plt.figure(figsize=(15,5))
x = np.arange(44, 180, 0.2)
data=np.array(unsuccessful_ts.iloc[0,220:900])
plt.plot(x,data)
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)
plt.xlabel('Time (seconds)', fontsize=10)
plt.ylabel('Current (nA)', fontsize=10)
plt.title('sensor noise in unsuccessful waveform', fontsize=15)
plt.show()

### Sample noise

In [ ]:
plt.style.use('default')
plt.figure(figsize=(15,5))
x = np.arange(200, 300.2, 0.2)
#print(x)
data=np.array(unsuccessful_ts.iloc[0,1000:])
plt.plot(x,data)
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)
plt.xlabel('Time (seconds)', fontsize=10)
plt.ylabel('Current (nA)', fontsize=10)
plt.title('Sample noise in unsuccessful waveform', fontsize=10)
plt.show()

### Hopefully, we need to smoothen these noise out

## Different filters:

**Low pass filter : Letting low frequencies pass ==> smoothening** <br>
**High pass filter : Letting high frequencies pass ==> sharpening** <br>
**Band pass filter : Letting frequencies of certain range to pass**

# Example of smoothening

![](smoothening.png)

# Low pass filter

In [ ]:
from scipy.signal import butter, lfilter, freqz
from matplotlib import pyplot as plt

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    print("normal cutoff is ",normal_cutoff)
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
order = 3
fs = 5 # sample rate, Hz
cutoff = 1 # desired cutoff frequency of the filter, Hz
data= np.array(unsuccessful_ts.iloc[0,190:900])
x=np.arange(38,180,0.2)
y = butter_lowpass_filter(data, cutoff, fs, order)
plt.figure(figsize=(15,5))
plt.plot(x[5:],data[5:],'g-')
plt.plot(x[5:],y[5:],'k-')
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)
plt.xlabel('Time (seconds)', fontsize=10)
plt.ylabel('Current (nA)', fontsize=10)
plt.title('Filtering sensor noise', fontsize=15)
plt.show()
rms = np.sqrt(mean_squared_error(data,y))
print("Root mean square value",rms)

In [ ]:
order = 3
fs = 5 # sample rate, Hz
cutoff = 2 # desired cutoff frequency of the filter, Hz
data= np.array(unsuccessful_ts.iloc[0,1000:1295])
x=np.arange(200,259,0.2)
y = butter_lowpass_filter(data, cutoff, fs, order)
plt.figure(figsize=(15,5))
plt.plot(x[5:],data[5:],'g-')
plt.plot(x[5:],y[5:],'k-')
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)
plt.xlabel('Time (seconds)', fontsize=10)
plt.ylabel('Current (nA)', fontsize=10)
plt.title('Filtering sample noise', fontsize=15)
plt.show()
rms = np.sqrt(mean_squared_error(data,y))
print("Root mean square value",rms)

# FIR filter - band pass filter

In [ ]:
from sklearn.metrics import mean_squared_error
from  scipy.signal import firwin,lfilter

In [ ]:
low = np.linspace(0.05e-10,10e-9,10)
high = np.linspace(25e-7,100e-9,10)
rmse_list=[]
lfilt_result_list=[]
x=np.arange(200,300.2,0.2)
data=np.array(unsuccessful_ts.iloc[0,1000:])
i=0
for low,high in zip(low,high):
    i+=1
    b = firwin(4, [low,high], width=0.9, pass_zero=False,fs=5)
    lfilt_result = lfilter(b, [1.0], data)
    lfilt_result_list.append(lfilt_result)
    plt.figure(figsize=(30,40))
    plt.subplot(6,2,i)
    plt.plot(x,data)
    plt.plot(x,lfilt_result,'k-')
    plt.yticks(fontsize=10)
    plt.xticks(fontsize=10)
    plt.xlabel('Time (seconds)', fontsize=10)
    plt.ylabel('Current (nA)', fontsize=10)
    plt.title('Smoothened waveform (unsuccessful)', fontsize=15)
    plt.subplots_adjust(hspace=0.9,wspace=0.35)    
    rms = np.sqrt(mean_squared_error(data, lfilt_result))
    rmse_list.append(rms)

In [ ]:
low_list = np.linspace(0.5,1,20)
high_list = np.linspace(1,2.4,20)
rmse_list=[]
lfilt_result_list=[]
data=np.array(unsuccessful_ts.iloc[0,200:900])
i=0
for low,high in zip(low_list,high_list):
    i+=1
    b = firwin(4, [low,high], width=0.2, pass_zero=False,fs=5)
    lfilt_result = lfilter(b,[1.0], data)
    lfilt_result_list.append(lfilt_result)
    plt.figure(figsize=(30,40))
    plt.subplot(20,2,i)
    plt.plot(data)
    plt.plot(lfilt_result,'w-')
    plt.yticks(fontsize=10)
    plt.xticks(fontsize=10)
    plt.xlabel('Time (seconds)', fontsize=15)
    plt.ylabel('Current (nA)', fontsize=15)
    plt.title('Smoothened waveform (unsuccessful)', fontsize=15)
    plt.subplots_adjust(hspace=0.9,wspace=0.35)    
    rms = np.sqrt(mean_squared_error(data, lfilt_result))
    rmse_list.append(rms)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(b)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.title('FIR filter', fontsize=25)
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(np.arange(1,len(rmse_list)+1),rmse_list,'k-',linewidth=4.0) 
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel('index', fontsize=15)
plt.ylabel('RMS of filtered signals and actual signal', fontsize=15)
plt.title('Comparison of RMS values', fontsize=25,loc='center')
plt.show()

In [ ]:
print("rms of different filtered signals and the actual signal \n",rmse_list)
print("mean of rms values:",np.mean(rmse_list))

In [ ]:
plt.figure(figsize=(30,20))
plt.plot(np.array(unsuccessful_ts.iloc[0,225:900]),'g-')
plt.plot(pd.Series(lfilt_result_list[3][5:]),'w-')

# Waveforms of successful data

In [ ]:
plt.style.use('default')
plt.figure(figsize=(10,5))
x = np.arange(44, 261, 0.2)
data=np.array(successful_ts.iloc[0,220:1305])
plt.plot(data)
plt.yticks(fontsize=5)
plt.xticks(fontsize=5)
plt.xlabel('Time (seconds)', fontsize=5)
plt.ylabel('Current (nA)', fontsize=5)
plt.title('Example waveform of successful data', fontsize=10)
plt.show()

### Zooming on to the sensor's noise

In [ ]:
plt.style.use('default')
plt.figure(figsize=(20,10))
data=np.array(successful_ts.iloc[0,220:900])
plt.plot(data)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel('Time (seconds)', fontsize=15)
plt.ylabel('Current (nA)', fontsize=15)
plt.title('sensor noise of successful readings', fontsize=25)
plt.show()

### Zooming on to the sample's noise

In [ ]:
plt.style.use('dark_background')
plt.figure(figsize=(20,10))
data=np.array(successful_ts.iloc[0,1000:])
plt.plot(data)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel('Time (seconds)', fontsize=15)
plt.ylabel('Current (nA)', fontsize=15)
plt.title('Sample noise', fontsize=25)
plt.show()

In [ ]:
order = 3
fs = 20 # sample rate, Hz
plt.style.use('default')
cutoff = 5.2 # desired cutoff frequency of the filter, Hz
x=np.arange(44,180,0.2)
data= np.array(successful_ts.iloc[0,220:900])
y = butter_lowpass_filter(data, cutoff, fs, order)
plt.figure(figsize=(15,5))
plt.plot(x[5:],data[5:],'g-')
plt.plot(x[5:],y[5:],'k-')
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)
plt.xlabel('Time (seconds)', fontsize=10)
plt.ylabel('Current (nA)', fontsize=10)
plt.title('Filtering sensor noise', fontsize=15)
plt.show()
rms = np.sqrt(mean_squared_error(data,y))
print("Root mean square value",rms)